## Osadzenia

W naszym poprzednim przykładzie operowaliśmy na wysokowymiarowych wektorach typu bag-of-words o długości `vocab_size`, a niskowymiarowe wektory reprezentacji pozycyjnej jawnie konwertowaliśmy na rzadką reprezentację one-hot. Taka reprezentacja one-hot nie jest efektywna pod względem pamięci. Dodatkowo, każde słowo jest traktowane niezależnie od innych, więc zakodowane w ten sposób wektory one-hot nie oddają semantycznych podobieństw między słowami.

W tej jednostce będziemy kontynuować eksplorację zbioru danych **News AG**. Na początek załadujmy dane i pobierzmy kilka definicji z poprzedniej jednostki.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### Czym jest embedding?

**Embedding** to sposób reprezentacji słów za pomocą gęstych wektorów o niższej liczbie wymiarów, które odzwierciedlają semantyczne znaczenie słowa. Później omówimy, jak tworzyć znaczące embeddingi słów, ale na razie potraktujmy embeddingi jako metodę redukcji wymiarowości wektora słowa.

Warstwa embedding przyjmuje słowo jako dane wejściowe i zwraca wektor wyjściowy o określonym rozmiarze `embedding_size`. W pewnym sensie jest to bardzo podobne do warstwy `Dense`, ale zamiast przyjmować wektor zakodowany w formie one-hot jako dane wejściowe, potrafi przyjąć numer słowa.

Korzystając z warstwy embedding jako pierwszej warstwy w naszej sieci, możemy przejść od modelu bag-of-words do modelu **embedding bag**, w którym najpierw zamieniamy każde słowo w naszym tekście na odpowiadający mu embedding, a następnie obliczamy pewną funkcję agregującą dla wszystkich tych embeddingów, na przykład `sum`, `average` lub `max`.

![Obraz przedstawiający klasyfikator embedding dla pięciu słów w sekwencji.](../../../../../lessons/5-NLP/14-Embeddings/images/embedding-classifier-example.png)

Nasza sieć neuronowa klasyfikatora składa się z następujących warstw:

* Warstwa `TextVectorization`, która przyjmuje ciąg znaków jako dane wejściowe i zwraca tensor z numerami tokenów. Określimy rozsądny rozmiar słownika `vocab_size` i pominiemy rzadziej używane słowa. Kształt danych wejściowych będzie wynosił 1, a kształt danych wyjściowych będzie równy $n$, ponieważ otrzymamy $n$ tokenów, z których każdy zawiera liczby od 0 do `vocab_size`.
* Warstwa `Embedding`, która przyjmuje $n$ liczb i redukuje każdą z nich do gęstego wektora o określonej długości (w naszym przykładzie 100). W ten sposób tensor wejściowy o kształcie $n$ zostanie przekształcony w tensor o kształcie $n\times 100$.
* Warstwa agregacji, która oblicza średnią tego tensora wzdłuż pierwszej osi, tj. oblicza średnią wszystkich $n$ tensorów wejściowych odpowiadających różnym słowom. Aby zaimplementować tę warstwę, użyjemy warstwy `Lambda` i przekażemy do niej funkcję obliczającą średnią. Dane wyjściowe będą miały kształt 100 i będą stanowiły numeryczną reprezentację całej sekwencji wejściowej.
* Ostateczna warstwa klasyfikatora liniowego `Dense`.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


W podsumowaniu, w kolumnie **kształt wyjścia**, pierwszym wymiarem tensora `None` jest rozmiar minibatcha, a drugim długość sekwencji tokenów. Wszystkie sekwencje tokenów w minibatchu mają różne długości. Omówimy, jak sobie z tym poradzić w następnej sekcji.

Teraz przejdźmy do trenowania sieci:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **Uwaga**: budujemy wektoryzator na podstawie podzbioru danych. Robimy to, aby przyspieszyć proces, co może skutkować sytuacją, w której nie wszystkie tokeny z naszego tekstu będą obecne w słowniku. W takim przypadku te tokeny zostaną pominięte, co może prowadzić do nieco niższej dokładności. Jednak w rzeczywistości podzbiór tekstu często daje dobrą ocenę słownictwa.


### Radzenie sobie z różnymi rozmiarami sekwencji zmiennych

Zrozummy, jak przebiega trening w minibatchach. W powyższym przykładzie tensor wejściowy ma wymiar 1, a my używamy minibatchy o długości 128, więc rzeczywisty rozmiar tensora wynosi $128 \times 1$. Jednak liczba tokenów w każdym zdaniu jest różna. Jeśli zastosujemy warstwę `TextVectorization` do pojedynczego wejścia, liczba zwróconych tokenów będzie różna, w zależności od sposobu tokenizacji tekstu:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


Jednak gdy zastosujemy wektoryzator do kilku sekwencji, musi on wygenerować tensor o prostokątnym kształcie, więc wypełnia nieużywane elementy tokenem PAD (który w naszym przypadku jest zerem):


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

Tutaj możemy zobaczyć osadzenia:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **Uwaga**: Aby zminimalizować ilość wypełnienia, w niektórych przypadkach warto posortować wszystkie sekwencje w zbiorze danych według rosnącej długości (a dokładniej, liczby tokenów). Dzięki temu każda mini-partia będzie zawierać sekwencje o podobnej długości.


## Semantyczne osadzenia: Word2Vec

W naszym poprzednim przykładzie warstwa osadzenia nauczyła się mapować słowa na reprezentacje wektorowe, jednak te reprezentacje nie miały znaczenia semantycznego. Byłoby dobrze nauczyć się takiej reprezentacji wektorowej, w której podobne słowa lub synonimy odpowiadają wektorom bliskim sobie pod względem jakiejś odległości wektorowej (na przykład odległości euklidesowej).

Aby to osiągnąć, musimy wstępnie wytrenować nasz model osadzenia na dużym zbiorze tekstów, korzystając z techniki takiej jak [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Opiera się ona na dwóch głównych architekturach, które służą do tworzenia rozproszonej reprezentacji słów:

 - **Ciągły worek słów** (Continuous bag-of-words, CBoW), gdzie trenujemy model, aby przewidywał słowo na podstawie otaczającego kontekstu. Mając n-gram $(W_{-2},W_{-1},W_0,W_1,W_2)$, celem modelu jest przewidzenie $W_0$ na podstawie $(W_{-2},W_{-1},W_1,W_2)$.
 - **Ciągły skip-gram** jest przeciwieństwem CBoW. Model wykorzystuje otaczające słowa z okna kontekstowego do przewidywania bieżącego słowa.

CBoW działa szybciej, natomiast skip-gram, choć wolniejszy, lepiej reprezentuje rzadkie słowa.

![Obraz przedstawiający algorytmy CBoW i Skip-Gram do konwersji słów na wektory.](../../../../../lessons/5-NLP/14-Embeddings/images/example-algorithms-for-converting-words-to-vectors.png)

Aby przeprowadzić eksperyment z osadzeniem Word2Vec wstępnie wytrenowanym na zbiorze danych Google News, możemy skorzystać z biblioteki **gensim**. Poniżej znajdujemy słowa najbardziej podobne do 'neural'.

> **Uwaga:** Przy pierwszym tworzeniu wektorów słów ich pobieranie może zająć trochę czasu!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Możemy również wyodrębnić osadzenie wektorowe z słowa, które będzie używane do trenowania modelu klasyfikacyjnego. Osadzenie ma 300 komponentów, ale tutaj pokazujemy tylko pierwsze 20 komponentów wektora dla przejrzystości:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Wspaniałą cechą osadzeń semantycznych jest możliwość manipulowania kodowaniem wektorowym na podstawie semantyki. Na przykład możemy poprosić o znalezienie słowa, którego reprezentacja wektorowa jest jak najbliższa słowom *król* i *kobieta*, a jak najdalsza od słowa *mężczyzna*:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Przykład powyżej wykorzystuje wewnętrzną magię GenSym, ale podstawowa logika jest w rzeczywistości dość prosta. Interesującą rzeczą dotyczącą osadzeń jest to, że można wykonywać normalne operacje wektorowe na wektorach osadzeń, a to odzwierciedla operacje na **znaczeniach** słów. Przykład powyżej można wyrazić w kategoriach operacji wektorowych: obliczamy wektor odpowiadający **KRÓL-MĘŻCZYZNA+KOBIETA** (operacje `+` i `-` są wykonywane na reprezentacjach wektorowych odpowiadających słów), a następnie znajdujemy najbliższe słowo w słowniku do tego wektora:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTE**: Musieliśmy dodać małe współczynniki do wektorów *man* i *woman* - spróbuj je usunąć, aby zobaczyć, co się stanie.

Aby znaleźć najbliższy wektor, używamy mechanizmów TensorFlow do obliczenia wektora odległości między naszym wektorem a wszystkimi wektorami w słowniku, a następnie znajdujemy indeks minimalnego słowa za pomocą `argmin`.


Chociaż Word2Vec wydaje się być świetnym sposobem na wyrażenie semantyki słów, ma wiele wad, w tym następujące:

* Zarówno modele CBoW, jak i skip-gram to **predykcyjne osadzenia** (predictive embeddings), które uwzględniają jedynie lokalny kontekst. Word2Vec nie wykorzystuje globalnego kontekstu.
* Word2Vec nie bierze pod uwagę **morfologii** słów, czyli faktu, że znaczenie słowa może zależeć od różnych części tego słowa, takich jak jego rdzeń.

**FastText** stara się przezwyciężyć drugie ograniczenie i rozwija Word2Vec, ucząc się reprezentacji wektorowych dla każdego słowa oraz n-gramów znakowych znajdujących się w obrębie tego słowa. Wartości tych reprezentacji są następnie uśredniane do jednego wektora na każdym kroku treningu. Chociaż dodaje to dużo dodatkowych obliczeń podczas wstępnego treningu, umożliwia osadzeniom słów kodowanie informacji o podczęściach słów.

Inna metoda, **GloVe**, wykorzystuje odmienne podejście do osadzeń słów, oparte na faktoryzacji macierzy słowo-kontekst. Najpierw buduje dużą macierz, która zlicza liczbę wystąpień słów w różnych kontekstach, a następnie próbuje przedstawić tę macierz w niższych wymiarach w sposób minimalizujący stratę rekonstrukcji.

Biblioteka gensim obsługuje te osadzenia słów, a Ty możesz eksperymentować z nimi, zmieniając kod ładowania modelu powyżej.


## Korzystanie z wstępnie wytrenowanych osadzeń w Keras

Możemy zmodyfikować powyższy przykład, aby wstępnie wypełnić macierz w naszej warstwie osadzeń semantycznymi osadzeniami, takimi jak Word2Vec. Słowniki wstępnie wytrenowanych osadzeń i korpusu tekstowego prawdopodobnie nie będą się pokrywać, więc musimy wybrać jeden z nich. Tutaj badamy dwie możliwe opcje: użycie słownika tokenizera oraz użycie słownika z osadzeń Word2Vec.

### Użycie słownika tokenizera

Kiedy korzystamy ze słownika tokenizera, niektóre słowa ze słownika będą miały odpowiadające im osadzenia Word2Vec, a niektórych będzie brakować. Zakładając, że rozmiar naszego słownika to `vocab_size`, a długość wektora osadzeń Word2Vec to `embed_size`, warstwa osadzeń będzie reprezentowana przez macierz wag o kształcie `vocab_size`$\times$`embed_size`. Wypełnimy tę macierz, przechodząc przez słownik:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


Dla słów, które nie występują w słowniku Word2Vec, możemy albo pozostawić je jako zera, albo wygenerować losowy wektor.

Teraz możemy zdefiniować warstwę osadzania z wstępnie wytrenowanymi wagami:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **Uwaga**: Zauważ, że ustawiliśmy `trainable=False` podczas tworzenia `Embedding`, co oznacza, że nie trenujemy ponownie warstwy Embedding. Może to spowodować nieznaczne obniżenie dokładności, ale przyspiesza proces uczenia.

### Korzystanie ze słownictwa embeddingu

Jednym z problemów w poprzednim podejściu jest to, że słownictwa używane w TextVectorization i Embedding są różne. Aby rozwiązać ten problem, możemy skorzystać z jednego z następujących rozwiązań:
* Ponowne przetrenowanie modelu Word2Vec na naszym słownictwie.
* Załadowanie naszego zbioru danych ze słownictwem pochodzącym z wstępnie wytrenowanego modelu Word2Vec. Słownictwo używane do ładowania zbioru danych można określić podczas ładowania.

Drugie podejście wydaje się prostsze, więc je zaimplementujmy. Na początek stworzymy warstwę `TextVectorization` ze zdefiniowanym słownictwem, pochodzącym z embeddingów Word2Vec:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

Biblioteka osadzania słów gensim zawiera wygodną funkcję `get_keras_embeddings`, która automatycznie utworzy dla Ciebie odpowiednią warstwę osadzania Keras.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


Jednym z powodów, dla których nie widzimy wyższej dokładności, jest brak niektórych słów z naszego zbioru danych w wstępnie wytrenowanym słowniku GloVe, przez co są one zasadniczo ignorowane. Aby to przezwyciężyć, możemy wytrenować własne osadzenia na podstawie naszego zbioru danych.


## Kontekstowe osadzenia

Jednym z kluczowych ograniczeń tradycyjnych wstępnie wytrenowanych reprezentacji osadzeń, takich jak Word2Vec, jest fakt, że choć potrafią one uchwycić pewne znaczenie słowa, nie są w stanie rozróżnić różnych znaczeń. Może to powodować problemy w modelach wykorzystujących te osadzenia.

Na przykład słowo „play” ma różne znaczenia w tych dwóch zdaniach:
- Poszedłem na **sztukę** do teatru.
- John chce się **bawić** ze swoimi przyjaciółmi.

Wstępnie wytrenowane osadzenia, o których mówiliśmy, reprezentują oba znaczenia słowa „play” w ten sam sposób. Aby przezwyciężyć to ograniczenie, musimy budować osadzenia oparte na **modelu językowym**, który jest trenowany na dużym korpusie tekstu i *wie*, jak słowa mogą być używane w różnych kontekstach. Omówienie kontekstowych osadzeń wykracza poza zakres tego samouczka, ale wrócimy do nich, gdy będziemy omawiać modele językowe w następnej jednostce.



---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż dokładamy wszelkich starań, aby zapewnić poprawność tłumaczenia, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za źródło autorytatywne. W przypadku informacji o kluczowym znaczeniu zaleca się skorzystanie z profesjonalnego tłumaczenia przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z użycia tego tłumaczenia.
